## Generating Flag Matroids

In this notebook, we determine all rank-$(1,2,3)$ flag matroids on $[4]$ up to $\mathfrak{S}_2 \times \mathfrak{S}_4$-symmetry. 

In [17]:
using Oscar

Polymake

**Function**: ```matroidsFromFile```

*Input*: ```fname = String, n = Int64```

*Output*: ```Dict{Int64,Vector{Any}}```

*Description*: Reads a file containing the matroids on $[n]$ where each line records a matroid as a string of ```*``` (bases) and ```0``` (nonbases) ordered by revlex. This function outputs a dictionary where the keys are the possible ranks, and the value at a rank ```d``` is the list of ($\mathfrak{S}_n$-orbit representatives of) $(d,n)$-matroids. 

---

**Function**: ```matroidsBasesFromFile```

*Input*: ```fname = String, n = Int64```

*Output*: ```Dict{Int64,Vector{Any}}```

*Description*: Reads a file containing the matroids on $[n]$ where each line records a matroid as a string of ```*``` (bases) and ```0``` (nonbases) ordered by revlex. This function outputs a dictionary where the keys are the possible ranks, and the value at a rank ```d``` is the list of the bases of ($\mathfrak{S}_n$-orbit representatives of) $(d,n)$-matroids. 

---

**Function**: ```permOnBases```

*Input*: ```Bs = Vector{Set{Int64}}, sigma = Permutation```

*Output*: ```Vector{Set{Int64}}```

*Description*: ```Bs``` is a list of bases, and this function permutes the elements within each basis by ```sigma```.

---

**Function**: ```set2Array```

*Input*: ```S = Set```

*Output*: ```Vector```

*Description*: Converts a set to a vector (elements in an arbitrary order)

---

**Function**: ```setBases2Matroid```

*Input*: ```Bs = Set{Set{Int64}}, n = Int64```

*Output*: ```Matroid```

*Description*: Given a set of bases ```Bs``` and $n$, returns the matroid on $[n]$ with these bases. 

---

**Function**: ```allMatroids```

*Input*: ```reps = Dict{Int64,Vector{Any}}, n = Int64```

*Output*: ```Dict{Int64,Any}```

*Description*: This fuunction takes as input $\mathfrak{S}_n$ orbit representatives of the matroids on $[n]$ as a dictionary like the output of ```matroidsBasesFromFile``` and returns all of the matroids on $[n]$ as a dictionary (keys = ranks, values=matroids of that rank).

---

**Function**: ```flatsMatroid```

*Input*: ```M=Matroid```

*Output*: ```Vector{Set{Int64}}```

*Description*: Given a matroid $M$ returns a list of its flats. 

---

**Function**: ```isQuotient```

*Input*: ```M,N = Matroid```

*Output*: ```true``` or ```false```

*Description*: returns ```true``` if ```M``` is a quotient of ```N``` and ```false``` otherwise. 

---

**Function**: ```flagMatroids```

*Input*: ```matroidReps = Dict{Int64,Vector{Any}}, n = Int64```

*Output*: ```Vector{Vector{Set{Set{Int64}}}}```

*Description*: Returns a list of the complete flag matroids on $[n]$. Each flag matroid is a list of matroids, each matroid is recorded as a list of its bases. 

---

**Function**: ```indicatorVector```

*Input*: ```S=Set```

*Output*: ```Vector{Int64}```

*Description*: Returns the indicator vector of the set ```S```.

---

**Function**: ```unionSets```

*Input*: ```Vector{Set{Int64}}```

*Output*: ```Set{Int64}```

*Description*: returns the union of a list of sets.

---

**Function**: ```flagBases2Flags```

*Input*: ```Bs = Vector{Set{Set{Int64}}}```

*Output*: ```Vector{Vector{Set{Int64}}}```

*Description*: Given a flag matroid (as a list of list of bases) returns all flags of bases. 

---

**Function**: ```flag2Permutation```

*Input*: ```Vector{Set{Int64}}```

*Output*: ```Vector{Int64}```

*Description*: Given a complete flag of subsets, returns its corresponding permutation (as a vector $(\sigma(1),\ldots, \sigma(n))$).

---

**Function**: ```flagBases2Permutations```

*Input*: ```Bs = Vector{Set{Set{Int64}}}, n = Int64```

*Output*: ```Set{Vector{Int64}}```

*Description*: Given a flag matroid (as a list of list of bases) returns the permutations of all flags from ```Bs```. The convex hull of these points is the flag matroid polytope. 

---

**Function**: ```dualFlagPermutations```

*Input*: ```Ps = Set{Vector{Int64}}, n = Int64```

*Output*: ```Set{Vector{Int64}}```

*Description*: Given a collection of permutations, returns the collection of their duals. 

---

**Function**: ```orbitFlagMatroidPolytope```

*Input*: ```Ps = Set{Vector{Int64}}, n = Int64```

*Output*: ```Set{Set{Vector{Int64}}}```

*Description*: Given a flag matroid as a set of the vertices in its polytope, returns all such flag matroids in its $(\mathfrak{S}_2 \times \mathfrak{S}_n)$-orbit.

---

**Function**: ```flagMatroidReps```

*Input*: ```Vector{Set{Vector{Int64}}}```

*Output*: ```Dict{Set{Vector{Int64}}, Set{Set{Vector{Int64}}}}```

*Description*: Given a collection of flag matroids, returns a dictionary whose keys are particular $\mathfrak{S}_2\times \mathfrak{S}_n$-orbit representatives of flag matroids, and the values are the corresponding $\mathfrak{S}_2\times \mathfrak{S}_n$-orbits. 

---

**Function**: ```vectors2Matrix```

*Input*: ```S = Set{Vector{Int64}}```

*Output*: ```Matrix{Int64}```

*Description*: Given a set of vectors of the same length, returns a matrix whose rows are these vectors (arbitrary order). 

---

**Function**: ```setPermutations2Polytope```

*Input*: ```S = Set{Vector{Int64}}, n = Int64```

*Output*: ```Polyhedron```

*Description*: Given a set of vectors of the same length, returns their convex hull as a polyhedron. 




In [18]:
function matroidsFromFile(fname, n)
    inf = open(fname);
    ls = readlines(inf);
    
    n2 = Int64(floor(n//2))
    
    Mats = Dict{Int64,Vector{Any}}(d => [] for d ∈ 1:n-1)
    
    for d ∈ 1:n2
        nCd = binomial(n,d)
        for st ∈ ls
            if length(st) == nCd
                append!(Mats[d], [pm.matroid.bases_from_revlex_encoding(st, d, n)])
                append!(Mats[n-d], [pm.matroid.bases_from_revlex_encoding(st, d, n, dual=true)])
            end
        end
    end
    
    return Mats
end

function matroidsBasesFromFile(fname, n)
    inf = open(fname);
    ls = readlines(inf);
    
    n2 = Int64(floor(n//2))
    
    Mats = Dict{Int64,Vector{Any}}(d => [] for d ∈ 1:n-1)
    
    for d ∈ 1:n2
        nCd = binomial(n,d)
        for st ∈ ls
            if length(st) == nCd                
                Bd = Vector{Set{Int64}}(pm.matroid.bases_from_revlex_encoding(st, d, n))
                Bnd = Vector{Set{Int64}}(pm.matroid.bases_from_revlex_encoding(st, d, n, dual=true))

                append!(Mats[d], [pm.to_one_based_indexing(Bd)]  )
                append!(Mats[n-d], [pm.to_one_based_indexing(Bnd)] )
            end
        end
    end
    
    return Mats
end


function permOnBases(Bs, sigma)
    return [on_sets(B, sigma) for B ∈ Bs] 
end

function matroidOrbit(Bs, n)
    orb = Set([])
    Sn = symmetric_group(n)
    SnElem = [sigma for sigma ∈ Sn]
    for sigma ∈ SnElem
        push!(orb, Set(permOnBases(Bs, sigma)))
    end
    return orb
end

function set2Array(S)
    return [i for i ∈ S]
end

function setBases2Matroid(Bs,n)
    BsVec = set2Array(Bs)
    return pm.matroid.Matroid(BASES = pm.to_zero_based_indexing(BsVec), N_ELEMENTS = n)
end


function allMatroids(reps, n)
    Ms = Dict{Int64,Any}(d => Set([]) for d ∈ 1:n-1)
    for d ∈ 1:n-1
        for M ∈ reps[d] 
            union!(Ms[d], matroidOrbit(M,n)) 
        end
    end
    return Ms
end

function flatsMatroid(M)
    LatticeFlatsM = M.LATTICE_OF_FLATS
    return Vector{Set{Int64}}(pm.@pm common.convert_to{Array{Set{Int}}}(LatticeFlatsM.FACES))
end

function isQuotient(M, N)
    FlatsM = Set{Set{Int64}}(flatsMatroid(M))
    FlatsN = Set{Set{Int64}}(flatsMatroid(N))
    return issubset(FlatsM, FlatsN)
end


function flagMatroids(matroidReps, n)
    Flags = Vector([[a] for a ∈ matroidReps[n-1]]);
    newFlags = Vector{Vector{Set{Set{Int64}}}}([])
    for i ∈ 2:n-1
        d = n-i
        for Bd in matroidReps[d]
            Md = pm.matroid.Matroid(BASES=pm.to_zero_based_indexing([a for a ∈ Bd]), N_ELEMENTS=n)
            for Fd1 ∈ Flags
                Bd1 = last(Fd1)
                Md1 = pm.matroid.Matroid(BASES=pm.to_zero_based_indexing([a for a ∈ Bd1]), N_ELEMENTS=n)
                if isQuotient(Md,Md1)
                    push!(newFlags, push!(Fd1[:], Bd))
                end
            end
        end
        Flags = newFlags
        newFlags =  Vector{Vector{Set{Set{Int64}}}}([])
    end
    return Flags
end

function indicatorVector(S,n)
    v = Vector{Int64}([])
    for i ∈ 1:n
        if i ∈ S
            push!(v,Int64(1))
        end
        if i ∉ S
            push!(v,Int64(0))
        end
    end
    return v
end

function unionSets(Ss)
    n = length(Ss)
    U = Set{Set{Int64}}([])
    for i ∈ 1:n
        union!(U,Ss[i])
    end
    return U
end

function flagBases2Flags(Bs)
    n = length(Bs)
    Flags = [[B] for B ∈ Bs[1]]
    newFlags = Vector{Vector{Set{Int64}}}([])
    for i ∈ 2:n
        for B ∈ Bs[i]
            for Fl ∈ Flags
                if issubset(B,last(Fl)) 
                    push!(newFlags, push!(Fl[:], B))
                end
            end
        end
        Flags = newFlags
        newFlags = Vector{Vector{Set{Int64}}}([])
    end
    return Flags
end

function flag2Permutation(Fl,n)
    return sum(map((x) -> indicatorVector(x,n), Fl)) + [Int64(1) for i ∈ 1:n ]
end

function flagBases2Permutations(Bs, n)
    Flags = flagBases2Flags(Bs)
    return Set{Vector{Int64}}(map((x) -> flag2Permutation(x,n), Flags))
end

function dualFlagPermutations(Ps, n)
    return Set([ (n+1)*[1 for i ∈ 1:n] - v for v ∈ Ps     ])
end

function orbitFlagMatroidPolytope(Ps, n)
    Sn = symmetric_group(n); SnElm = [s for s ∈ Sn];
    PsArr = set2Array(Ps)
    Orb = Set{Set{Vector{Int64}}}([Set{Vector{Int64}}(Set(map((x) -> x[Vector(sigma)], PsArr))) for sigma ∈ SnElm])
    Orb2 = Set{Set{Vector{Int64}}}( [dualFlagPermutations(p,n) for p ∈ Orb]  )
    return union!(Orb,Orb2)
end


function flagMatroidReps(allPs, n)
    reps = Dict{Set{Vector{Int64}}, Set{Set{Vector{Int64}}}}([])
    for Ps ∈ allPs
        Orb = orbitFlagMatroidPolytope(Ps, n)
        if Orb ∉ values(reps)
            reps[Ps] = Orb
        end
    end
    return reps
end

function vectors2Matrix(S)
    vs = [i for i ∈ v, v ∈ S]
end
    

function setPermutations2Polytope(S,n)
    vs = [v[i] for v ∈ S, i ∈ 1:n]; 
    return convex_hull(vs)
end


flagMatroids (generic function with 1 method)

In [9]:



function maxRep(Bs)
    
    
    
end


setPermutations2Polytope (generic function with 1 method)

In [25]:
orbitFlagMatroidPolytope(allPolytopes4[1], 4)

Set{Set{Vector{Int64}}} with 3 elements:
  Set([[4, 3, 2, 1], [2, 1, 4, 3], [2, 4, 1, 3], [3, 1, 4, 2], [2, 3, 1, 4], [4…
  Set([[2, 1, 4, 3], [4, 3, 2, 1], [1, 3, 4, 2], [3, 2, 1, 4], [3, 4, 1, 2], [1…
  Set([[1, 3, 4, 2], [3, 2, 1, 4], [2, 4, 1, 3], [3, 1, 4, 2], [4, 1, 3, 2], [2…

In [7]:
n4 = matroidsBasesFromFile("matroidDatabase/n4.txt",4);
alln4 = allMatroids(n4, 4);
fl4 = flagMatroids(alln4,4);
allPolytopes4 = map((x) -> flagBases2Permutations(x,4), fl4)
repsDict4 = flagMatroidReps(allPolytopes4,4)
reps4 = keys(repsDict4)

In [26]:
allPolytopes4

406-element Vector{Set{Vector{Int64}}}:
 Set([[2, 1, 4, 3], [4, 3, 2, 1], [3, 1, 4, 2], [4, 1, 3, 2], [2, 4, 1, 3], [2, 3, 1, 4], [3, 4, 1, 2], [1, 2, 3, 4], [3, 4, 2, 1], [2, 1, 3, 4], [3, 2, 4, 1], [4, 2, 3, 1], [1, 4, 2, 3], [1, 3, 2, 4], [1, 2, 4, 3], [4, 3, 1, 2]])
 Set([[4, 3, 2, 1], [3, 4, 2, 1], [3, 2, 4, 1], [4, 2, 3, 1], [1, 4, 2, 3], [1, 3, 2, 4], [1, 2, 4, 3], [1, 2, 3, 4]])
 Set([[2, 1, 4, 3], [2, 1, 3, 4], [3, 1, 4, 2], [4, 1, 3, 2], [2, 4, 1, 3], [2, 3, 1, 4], [4, 3, 1, 2], [3, 4, 1, 2]])
 Set([[2, 1, 4, 3], [2, 1, 3, 4], [1, 4, 2, 3], [1, 3, 2, 4], [1, 2, 4, 3], [4, 1, 2, 3], [1, 2, 3, 4], [3, 1, 2, 4]])
 Set([[2, 1, 4, 3], [3, 2, 1, 4], [2, 4, 1, 3], [2, 3, 1, 4], [4, 1, 2, 3], [1, 2, 3, 4], [4, 2, 1, 3], [2, 1, 3, 4], [1, 4, 2, 3], [1, 3, 2, 4], [1, 2, 4, 3], [3, 1, 2, 4]])
 Set([[2, 1, 4, 3], [4, 2, 1, 3], [2, 1, 3, 4], [3, 2, 1, 4], [2, 4, 1, 3], [2, 3, 1, 4], [4, 1, 2, 3], [3, 1, 2, 4]])
 Set([[4, 2, 1, 3], [3, 2, 1, 4], [2, 4, 1, 3], [1, 4, 2, 3], [2, 3, 1, 4], [1

In [16]:
Dims4 = Dict{Set{Vector{Int64}}, Int64}([ x => dim(setPermutations2Polytope(x,4)) for x in reps4   ])
[x for x in reps4 if Dims4[x] == 3 ]

15-element Vector{Set{Vector{Int64}}}:
 Set([[4, 3, 2, 1], [4, 1, 3, 2], [4, 1, 2, 3], [2, 4, 3, 1], [1, 2, 3, 4], [1, 4, 3, 2], [3, 4, 2, 1], [2, 1, 3, 4], [4, 2, 3, 1], [1, 4, 2, 3], [1, 3, 2, 4], [3, 1, 2, 4]])
 Set([[2, 1, 4, 3], [3, 2, 1, 4], [3, 1, 4, 2], [4, 1, 3, 2], [2, 4, 1, 3], [2, 3, 1, 4], [4, 1, 2, 3], [2, 4, 3, 1], [4, 2, 1, 3], [2, 1, 3, 4], [3, 2, 4, 1], [4, 2, 3, 1], [2, 3, 4, 1], [3, 1, 2, 4]])
 Set([[2, 1, 4, 3], [1, 3, 4, 2], [3, 1, 4, 2], [4, 1, 3, 2], [4, 1, 2, 3], [1, 2, 3, 4], [1, 4, 3, 2], [2, 1, 3, 4], [1, 4, 2, 3], [1, 3, 2, 4], [1, 2, 4, 3], [3, 1, 2, 4]])
 Set([[4, 2, 1, 3], [2, 1, 3, 4], [3, 2, 1, 4], [4, 2, 3, 1], [4, 1, 3, 2], [2, 4, 1, 3], [2, 3, 1, 4], [4, 1, 2, 3], [2, 4, 3, 1], [3, 1, 2, 4]])
 Set([[2, 1, 4, 3], [2, 1, 3, 4], [1, 4, 2, 3], [1, 3, 2, 4], [1, 2, 4, 3], [4, 1, 2, 3], [1, 2, 3, 4], [3, 1, 2, 4]])
 Set([[4, 3, 2, 1], [3, 4, 2, 1], [3, 2, 4, 1], [4, 2, 3, 1], [1, 4, 2, 3], [1, 3, 2, 4], [1, 2, 4, 3], [1, 2, 3, 4]])
 Set([[2, 1, 4, 3], [4,